In [1]:
#@title координаты зон обзора
import json
import math

def calculate_view_zone(lat, lng, azimuth, radius_meters):
    """
    Рассчитывает координаты границ сектора обзора камеры
    """
    # Конвертация углов в радианы
    left_angle = math.radians(azimuth - 30)
    right_angle = math.radians(azimuth + 30)

    # Преобразование метров в градусы
    lat_degree_meters = 111320  # метров в одном градусе широты
    lng_degree_meters = 111320 * math.cos(math.radians(lat))  # метров в одном градусе долготы

    lat_delta = radius_meters / lat_degree_meters
    lng_delta = radius_meters / lng_degree_meters

    # Левая граница (левый угол обзора)
    lat_1 = lat + lat_delta * math.cos(left_angle)
    lng_1 = lng + lng_delta * math.sin(left_angle)

    # Правая граница (правый угол обзора)
    lat_2 = lat + lat_delta * math.cos(right_angle)
    lng_2 = lng + lng_delta * math.sin(right_angle)

    return lat_1, lng_1, lat_2, lng_2

def process_cameras(input_file, output_file):
    """
    Обрабатывает все камеры из входного файла и сохраняет результаты
    """
    # Загрузка данных о камерах
    with open(input_file, 'r', encoding='utf-8') as f:
        cameras_data = json.load(f)

    # Обработка каждой камеры
    processed_cameras = []

    for camera in cameras_data:
        # Определение радиуса обзора в зависимости от типа камеры
        camera_type = camera.get('cameraType', '')
        if camera_type.startswith('1'):
            radius = 10  # 10 метров для камер типа 1
        else:
            radius = 25  # 25 метров для других типов

        # Расчет зоны обзора
        lat_1, lng_1, lat_2, lng_2 = calculate_view_zone(
            camera['lat'],
            camera['lng'],
            camera['defaultAzimuth'],
            radius
        )

        # Создание нового объекта с дополнительными данными (без viewRadius)
        processed_camera = {
            "name": camera['name'],
            "lat": camera['lat'],
            "lng": camera['lng'],
            "district": camera['district'],
            "address": camera['address'],
            "cameraType": camera['cameraType'],
            "defaultAzimuth": camera['defaultAzimuth'],
            "lat_1": lat_1,
            "lng_1": lng_1,
            "lat_2": lat_2,
            "lng_2": lng_2
        }

        processed_cameras.append(processed_camera)

    # Сохранение результатов в файл
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_cameras, f, ensure_ascii=False, indent=2)

    print(f"Обработано {len(processed_cameras)} камер")
    print(f"Результаты сохранены в файл: {output_file}")

# Основной код
if __name__ == "__main__":
    input_file = '/content/drive/MyDrive/useful/v_json/v_name.json'
    output_file = '/content/drive/MyDrive/useful/intersections/Actual/zones.json'

    try:
        process_cameras(input_file, output_file)
        print("Обработка завершена успешно!")
    except FileNotFoundError:
        print(f"Ошибка: файл {input_file} не найден")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

Обработано 94357 камер
Результаты сохранены в файл: /content/drive/MyDrive/useful/intersections/Actual/zones.json
Обработка завершена успешно!
